In [1]:
from pyspark.sql import SparkSession

# create a SparkSession
spark = SparkSession.builder.appName("ReadJSON").config("spark.executor.memory", "4g").config("spark.driver.memory", "4g").getOrCreate()


24/04/13 18:11:42 WARN Utils: Your hostname, albertastein-2.local resolves to a loopback address: 127.0.0.1; using 10.0.0.76 instead (on interface en0)
24/04/13 18:11:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/13 18:11:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType, BooleanType, IntegerType
#"reviewerID": "A8WEXFRWX1ZHH", 
# "asin": "0209688726", 
# "style": {"Color:": " AC"}, 
# "reviewerName": "Goldengate",
# Define the schema
schema = StructType([
    StructField("overall", FloatType(), True),
    StructField("verified", BooleanType(), True),
    StructField("reviewTime", StringType(), True),
    StructField("reviewerID", StringType(), True),
    StructField("asin", StringType(), True),
    StructField("style", StructType([StructField("Color:", StringType(), True)]), True),
    StructField("reviewerName", StringType(), True),
    StructField("reviewText", StringType(), True),
    StructField("unixReviewTime", IntegerType(), True)
    
])


json_df = spark.read.schema(schema).json("combined_train_data_chunked_10mb_latest.json")
json_df.show(5)

# 

+-------+--------+-----------+--------------+----------+------+--------------+--------------------+--------------+
|overall|verified| reviewTime|    reviewerID|      asin| style|  reviewerName|          reviewText|unixReviewTime|
+-------+--------+-----------+--------------+----------+------+--------------+--------------------+--------------+
|    5.0|    true| 04 5, 2016|A1274GG1EB2JLJ|0486427706|{NULL}|   barbara ann|The pictures are ...|    1459814400|
|    5.0|    true|02 13, 2016|A30X5EGBYAZQQK|0486427706|{NULL}|      Samantha|I absolutely love...|    1455321600|
|    5.0|    true|12 10, 2015|A3U6UNXLAUY6ZV|0486427706|{NULL}|   CP in Texas|          I love it!|    1449705600|
|    5.0|    true|10 26, 2015|A1SAJF5SNM6WJS|0486427706|{NULL}|   LOIS LABIER|MY HUSBAND LOVED ...|    1445817600|
|    4.0|    true|09 15, 2015| AHJWO3SI0S0OR|0486427706|{NULL}|Saundra Hatley|                cool|    1442275200|
+-------+--------+-----------+--------------+----------+------+--------------+--

In [11]:
# Print the number of entries in the dataframe
print("Number of entries in the dataframe before pre processing: ", json_df.count())

Number of entries in the dataframe before pre processing:  11321389


In [5]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, FloatType, BooleanType, IntegerType, ArrayType
# targetUDF = F.udf(lambda x: 1 if x >= 4.0 else (0 if x == 3.0 else -1), IntegerType())
targetUDF = F.udf(lambda x: 1 if x >= 4.0 else 0, IntegerType())
import re
import nltk
from nltk.corpus import stopwords

In [7]:
reduced_df = json_df.select("overall", "reviewerID", "asin", "reviewText")



In [12]:
reduced_df.show(5)

+-------+--------------+----------+--------------------+
|overall|    reviewerID|      asin|          reviewText|
+-------+--------------+----------+--------------------+
|    5.0|A1274GG1EB2JLJ|0486427706|The pictures are ...|
|    5.0|A30X5EGBYAZQQK|0486427706|I absolutely love...|
|    5.0|A3U6UNXLAUY6ZV|0486427706|          I love it!|
|    5.0|A1SAJF5SNM6WJS|0486427706|MY HUSBAND LOVED ...|
|    4.0| AHJWO3SI0S0OR|0486427706|                cool|
+-------+--------------+----------+--------------------+
only showing top 5 rows



In [10]:
unique_df = reduced_df.dropDuplicates(["reviewerID", "asin"])
print("Number of entries in the dataframe after removing duplicates: ", unique_df.count())

Number of entries in the dataframe after removing duplicates:  10936206


In [9]:
def preProcess(text):
    # Should return a list of tokens
    text = re.sub(r"(\w)([.,;:!?'\"”\)])", r"\1 \2", text)
    text = re.sub(r"([.,;:!?'\"“\(])(\w)", r"\1 \2", text)    
    text = text.lower()
    tokens = word_tokenize(text)    
    return tokens

In [13]:
df_sentiment = unique_df.withColumn("sentiment", targetUDF(unique_df["overall"]))

In [14]:
df_sentiment.show()

/opt/homebrew/Cellar/apache-spark/3.5.1/libexec/python/lib/pyspark.zip/pyspark/daemon.py:154: DeprecationWarning: This process (pid=65340) is multi-threaded, use of fork() may lead to deadlocks in the child.


+-------+--------------------+----------+--------------------+---------+
|overall|          reviewerID|      asin|          reviewText|sentiment|
+-------+--------------------+----------+--------------------+---------+
|    5.0|A0015332H21AK8WZ0ZCS|B005G030TC|These collars are...|        1|
|    4.0| A0020356UF96ZV361ST|B00ZI5OVFM|It is a love stor...|        1|
|    2.0| A0020356UF96ZV361ST|B015X7KEDM|This book is not ...|        0|
|    5.0|A0024936S1WI02OHH9DP|B016AG5DR2|Looks great fits ...|        1|
|    5.0|A0034986DWR7WEDQN0GV|B001VJZO2S|           excellent|        1|
|    3.0|A00463782V7TKAP9EMNL|B002B3YBSQ|What a sad thing ...|        0|
|    4.0|A00463782V7TKAP9EMNL|B003SNJYOO|You may love some...|        1|
|    3.0|A00463782V7TKAP9EMNL|B00LB8LSIS|If he had any kin...|        0|
|    5.0|A00463782V7TKAP9EMNL|B00WLD2OQ2|K. Sterling has w...|        1|
|    3.0|A00463782V7TKAP9EMNL|B00ZAUVLOY|Two young men are...|        0|
|    5.0|A00463782V7TKAP9EMNL|B016XD4RH4|It's tough

In [16]:
from pyspark.ml.feature import Tokenizer

# use PySparks build in tokenizer to tokenize tweets
tokenizer = Tokenizer(inputCol  = "reviewText",
                      outputCol = "token")
# Remove the rows with missing values and tokenize
df_tokenized = tokenizer.transform(df_sentiment.filter(unique_df.reviewText.isNotNull()))

In [17]:

df_tokenized.show(5)

+-------+--------------------+----------+--------------------+---------+--------------------+
|overall|          reviewerID|      asin|          reviewText|sentiment|               token|
+-------+--------------------+----------+--------------------+---------+--------------------+
|    5.0|A0015332H21AK8WZ0ZCS|B005G030TC|These collars are...|        1|[these, collars, ...|
|    4.0| A0020356UF96ZV361ST|B00ZI5OVFM|It is a love stor...|        1|[it, is, a, love,...|
|    2.0| A0020356UF96ZV361ST|B015X7KEDM|This book is not ...|        0|[this, book, is, ...|
|    5.0|A0024936S1WI02OHH9DP|B016AG5DR2|Looks great fits ...|        1|[looks, great, fi...|
|    5.0|A0034986DWR7WEDQN0GV|B001VJZO2S|           excellent|        1|         [excellent]|
+-------+--------------------+----------+--------------------+---------+--------------------+
only showing top 5 rows



In [21]:
import re

def removeRegex(tokens: list) -> list:
    """
    Removes hashtags, call outs and web addresses from tokens.
    """
    # Use a raw string for regular expressions to avoid escape sequence warnings
    expr = r'(@[A-Za-z0-9_]+)|(#[A-Za-z0-9_]+)|'+\
           r'(https?://[^\s<>"]+|www\.[^\s<>"]+)'
    regex = re.compile(expr)
    cleaned = [t for t in tokens if not regex.search(t) and len(t) > 0]

    return cleaned


In [22]:
removeWEBUDF = F.udf(removeRegex, ArrayType(StringType()))

In [23]:
def normalize(tokens : list) -> list:
    """
    Removes non-english characters and returns lower case versions of words.
    """
    subbed   = [re.sub("[^a-zA-Z]+", "", s).lower() for s in tokens]
    
    filtered = filter(None, subbed)
    
    return list(filtered)


normalizeUDF = F.udf(normalize, ArrayType(StringType()))

In [24]:
# remove hashtags, call outs and web addresses
df4 = df_tokenized.withColumn("tokens_re", removeWEBUDF(df_tokenized["token"]))

# remove non english characters
df4 = df4.withColumn("tokens_clean", normalizeUDF(df4["tokens_re"]))

# rename columns
df5 = df4.drop("token","tokens_re")
df5 = df5.withColumnRenamed("tokens_clean", "tokens")\

# remove reviews where the tokens array is empty, i.e. where it was just
# a hashtag, callout, numbers, web adress etc.
df6 = df5.where(F.size(F.col("tokens")) > 0)

In [26]:
df_for_model = df6.select("reviewText","sentiment")\
        .withColumnRenamed("sentiment", "label")

In [30]:
from pyspark.sql.functions import rand

# Assuming 'df' is your DataFrame
shuffled_df = df_for_model.orderBy(rand())

# Show the shuffled DataFrame
shuffled_df.show()

+--------------------+-----+
|          reviewText|label|
+--------------------+-----+
|I purchased this ...|    1|
|I always keep one...|    1|
|I only have one p...|    1|
|Not a perfect fit...|    1|
|Great sander for ...|    1|
|have this in the ...|    1|
|so far so good.  ...|    1|
|Many great songs ...|    1|
|The ship is great...|    1|
|                good|    1|
|Son didn't like i...|    0|
|I bought the orig...|    1|
|I live way up in ...|    1|
|I was hoping that...|    0|
|The album has som...|    1|
|Very useful for s...|    1|
|I love this showe...|    1|
|This is a magneti...|    1|
|My partner report...|    1|
|worked fine for m...|    1|
+--------------------+-----+
only showing top 20 rows

